# CMIP Future Realizations: Drought Targets, SPEI, and SPI

The purpose of this notebook is to determine the probabilities, calculated from 2031-2060 CMIP fitted log logistic distributions, for drought target, 3-month cumulative deficit values.

SPEI and SPI, calculated using 2031-2060 CMIP, is also plotted for the future realizations.

## Parameters and Imports

In [1]:
%matplotlib inline

In [2]:
import os
from IPython.display import display, HTML
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import stats as sstats
from math import exp
import datetime as dt
import seaborn as sns
import pickle
from copy import deepcopy

In [3]:
plt.rcParams['svg.fonttype'] = 'none'

In [4]:
IN_DIR1 = r'C:\Users\nmartin\Documents\EAA_HCP\Data\SwRI_Processed\Processed_Historical'
IN_DIR2 = r'C:\Users\nmartin\Documents\EAA_HCP\Data\SwRI_Processed\DayMet_GridIntersect'
IN_DIR3 = r'C:\Users\nmartin\Documents\EAA_HCP\Data\SwRI_Processed\CMIP6'
IN_DIR4 = r'C:\Users\nmartin\Documents\EAA_HCP\Data\SwRI_Processed\CMIP5'
OUT_DIR = r'C:\Users\nmartin\Documents\EAA_HCP\Data\SwRI_Processed\CombinedFuture\Drought_Targets'

In [5]:
# our standard normal or ZScore parameters
ZMu = 0.0
ZStd = 1.0

In [6]:
# full basin intersection dictionary
InFiler = os.path.normpath( os.path.join( IN_DIR2, "BasWeightsGDF.pkl" ) )
with open( InFiler, 'rb' ) as IF:
    BasWeightsDF = pickle.load( IF )
# end with
BAS_KEYS = sorted( BasWeightsDF.keys() )

In [7]:
MonDict = { 1 : "Jan", 2 : "Feb", 3 : "Mar", 4 : "Apr", 5 : "May", 6 : "Jun", 7 : "Jul", 8 : "Aug",
            9 : "Sep", 10 : "Oct", 11 : "Nov", 12 : "Dec", }
MonKeys = sorted( MonDict.keys() )

In [8]:
MODEL_DICT = { 2 : ["CMIP6", ["ssp245", "ssp585"], ], 
               3 : ["CMIP6", ["ssp245", "ssp585"], ], 
               4 : ["CMIP6", ["ssp245", "ssp585"], ], 
               5 : ["CMIP6", ["ssp245", "ssp585"], ], 
               11 : ["CMIP6", ["ssp245", "ssp585"], ], 
               12 : ["CMIP6", ["ssp245", "ssp585"], ], 
               7 : ["CMIP5", ["rcp45", "rcp85"], ], 
               8 : ["CMIP5", ["rcp45", "rcp85"], ], 
               9 : ["CMIP5", ["rcp45", "rcp85"], ], 
               10 : ["CMIP5", ["rcp85"], ], }
MODEL_KEYS = list( MODEL_DICT.keys() )

In [9]:
MonIndexer = [ x for x in range(1,13,1)]

In [10]:
PTSStart = pd.Timestamp( 1993, 1, 1, 0 )
PTSStop = pd.Timestamp( 2022, 12, 31, 23, 59, )
CNTSStart = pd.Timestamp( 2031, 1, 1, 0 )
CNTSStop = pd.Timestamp( 2060, 12, 31, 23, 59, )

In [11]:
ProjModelOutsName = "Mod_%02d_%s_2023through2080_%s_DF.pkl"

In [12]:
# lambdas
setYrMo = lambda yr, mo: ( int(yr) * 100 ) + int(mo)
calcYr = lambda indx: int( indx / 100 )
calcMo = lambda indx, yr: int( indx - ( yr * 100 ) )

**start skip**

In [13]:
#InFiler = os.path.normpath( os.path.join( IN_DIR1, "FBas_MetTS_1980through2020_Dict.pkl" ) )
#with open( InFiler, 'rb' ) as IF:
#    OrgBasinsDFDict = pickle.load( IF )
## end with

In [14]:
#InFiler = os.path.normpath( os.path.join( IN_DIR1, "FBas_MetTS_2021through2022_Dict.pkl" ) )
#with open( InFiler, 'rb' ) as IF:
#    ExBasinsDFDict = pickle.load( IF )
## end with

Combine the observed into single DataFrames

In [15]:
#BasinsDFDict = dict()

In [16]:
#for bas in BAS_KEYS:
#    medDF = OrgBasinsDFDict[bas]
#    newMedDF = ExBasinsDFDict[bas]
#    fullMedDF = pd.concat( [ medDF, newMedDF ], )
#    BasinsDFDict[bas] = fullMedDF.copy()
## end for

In [17]:
# output for future
#OutFiler = os.path.normpath( os.path.join( IN_DIR1, "FBas_MetTS_1980through2022_Dict.pkl" ) )
#with open( OutFiler, 'wb' ) as OF:
#    pickle.dump( BasinsDFDict, OF, protocol=pickle.HIGHEST_PROTOCOL )
## end with

**end skip**

In [18]:
# load full time series
InFiler = os.path.normpath( os.path.join( IN_DIR1, "FBas_MetTS_1980through2022_Dict.pkl" ) )
with open( InFiler, 'rb' ) as IF:
    BasinsDFDict = pickle.load( IF )
# end with

## Custom Functions

In [19]:
def estimatellogparams( npArray ):
    """Estimate the parameters of a log-logistic distribution from an
    array of annual values.
    
    Estimate is done using L-moments and the "Generalized logistic distribution".
    This distribtion is a reparameterized version of the log-logistic
    distribution of Ahmad et al. (1988). Estimation is done using 
    the equations and procedure in Appendix A.7 of "Regional Frequency
    Analysis", Hosking and Wallis (1997)
    
    To estimate the distribution parameters (shape, scale, and location),
    the L-moments l1, l2, and t3 need to be calculated. These three
    L-moments can be estimated from the first three, sample weighted
    probability moments (b0, b1, and b2).
    
    Args:
        npArray (np.ndarray): Numpy, 1D array
    
    Returns:
        log-logistic parameters in dictionary, D:
            D["k"]: k or shape
            D["scale"]: alpha or scale
            D["loc"]: Eta or location
    """
    # imports
    import math
    # don't do any checking for type and assume that will always
    #  be Numpy ndarray for single argument
    totLen = len( npArray )
    # need a sorted array in increasing order
    srtAr = np.sort( npArray )
    # calculate sample probability weighted moments: b0, b1, b2
    b0 = srtAr.mean()
    b1 = 0.0
    for iI in range(2, totLen + 1):
        b1 += ( ( iI - 1 ) / ( totLen - 1 ) ) * srtAr[iI-1]
    # end for
    b1 = b1 / totLen
    b2 = 0.0
    for iI in range( 3, totLen + 1 ):
        b2 += ( ( ( iI - 1 ) * ( iI - 2 ) ) / ( ( totLen - 1 ) * (totLen - 2 ) ) ) * srtAr[iI-1]
    # end for
    b2 = b2 / totLen
    # calculate sample L-moments: l1, l2, t3
    l1 = b0
    l2 = (2.0 * b1 ) - b0
    l3 = ( 6.0 * b2 ) -  ( 6.0 * b1 ) + b0
    t3 = l3 / l2
    # estimate the distribution parameters
    shape = -1.0 * t3
    scale = ( l2 * math.sin( shape * math.pi ) ) / ( shape * math.pi )
    location = l1 - ( scale * ( ( 1.0 / shape ) - ( math.pi / math.sin( shape * math.pi ) ) ) )
    retDict = { "k" : shape,
                "scale" : scale,
                "loc" : location, }
    # return
    return retDict

In [20]:
def probDistLLogis( paramDict, npArray ):
    """Uses generalized logistic probability distribution to estimate cumulative
    probilities for each value in the Numpy array, npArray.
    
    Args:
        paramDict (dict): dictionary with best-fit parameter values for a 
                log-logisitic distribution. Must have keys: "k", "scale",
                "loc" which are the 3 required parameters
        npArray (np.ndarray): array from time series of monthly, rolling
                average values
    
    Returns:
        retArray (np.ndarray): cumulative probabilies for each npArray value
    """
    shape = paramDict["k"]
    location = paramDict["loc"]
    scale = paramDict["scale"]
    if shape == 0.0:
        # this is the special case of a logistic distribution with 2 params
        y = ( npArray - location ) / scale
    else:
        # this is the general case of the log-logistic distribution
        takeLogArray = 1.0 - ( shape * ( npArray - location ) / scale )
        useLogArray = np.where( takeLogArray <= 0.0, 1e-7, takeLogArray )
        y = ( -1.0 * ( 1.0 / shape ) ) * np.log( useLogArray )
    # end if
    retArray = 1.0 / ( 1.0 + np.exp( -1.0 * y  ) )
    # return
    return retArray

## Load the Drought Targets for Each Basin

In [21]:
DroughtTargsDict = dict()

In [22]:
for bas in BAS_KEYS:
    InFiler = os.path.normpath( os.path.join( IN_DIR1, "SPEI", bas, "%s_drought_targets.xlsx" % bas ) )
    curDF = pd.read_excel( InFiler, sheet_name="Targets", header=0, index_col=0, )
    cumDefTarg = float( curDF.at[7, "2022 Cum Def"] )
    cumProbOrig = float( curDF.at[7, "2022 Cum Prob"] )
    DroughtTargsDict[bas] = [ 7, cumProbOrig, cumDefTarg, dict() ]
# end for

## Get the Observed, Cumulative Deficits and Precip

In [23]:
ObsCumDef = dict()
ObsCumPre = dict()

In [24]:
for bas in BAS_KEYS:
    curDF = BasinsDFDict[bas]
    MonDF = curDF.resample( 'MS', ).sum()
    PreMonDF = MonDF[["Pre_mm"]].copy()
    DefMonDF = MonDF[["Def_mm"]].copy()
    P3DF = PreMonDF.rolling(window=3,).sum()
    D3DF = DefMonDF.rolling(window=3,).sum()
    P3DF["Month"] = P3DF.index.month
    D3DF["Month"] = D3DF.index.month
    exP3DF = P3DF.loc[PTSStart:PTSStop].copy()
    exD3DF = D3DF.loc[PTSStart:PTSStop].copy()
    m3Mon = exP3DF[exP3DF["Month"] == 1].copy()
    a3Mon = m3Mon["Pre_mm"].to_numpy(dtype=np.float32)
    ObsCumPre[bas] = { 1 : a3Mon, }
    m3Mon = exD3DF[exD3DF["Month"] == 1].copy()
    a3Mon = m3Mon["Def_mm"].to_numpy(dtype=np.float32)
    ObsCumDef[bas] = { 1 : a3Mon, }
    for mon in range(2,13):
        m3Mon = exP3DF[exP3DF["Month"] == mon].copy()
        a3Mon = m3Mon["Pre_mm"].to_numpy(dtype=np.float32)
        ObsCumPre[bas][mon] = a3Mon
        m3Mon = exD3DF[exD3DF["Month"] == mon].copy()
        a3Mon = m3Mon["Def_mm"].to_numpy(dtype=np.float32)
        ObsCumDef[bas][mon] = a3Mon
    # end month for
# end basin for

## SPEI and SPI with Plots

Do the SPEI calculation for each basin and model and scenario pair. Calculate the SPEI for 2031 through 2060.

In [25]:
pMax = 3.0
pMin = -3.0

In [26]:
cColors = [ "darkred", "firebrick", "indianred", "lightcoral", "lightsalmon", "antiquewhite",
            "antiquewhite", "antiquewhite", "lightblue", "cadetblue", "royalblue", "mediumblue", "midnightblue" ]
# -3.0, -2.5, -2.0, -1.5, -1.0, -0.5, 0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0
cNodes = [0.0, 0.0833, 0.1666, 0.25, 0.3333, 0.4166, 0.50, 0.5833, 0.666, 0.75, 0.8333, 0.9166, 1.0 ]
len(cColors), len(cNodes)

(13, 13)

In [27]:
SegCMap = mpl.colors.LinearSegmentedColormap.from_list("spei_cmap", list(zip(cNodes, cColors)))

In [28]:
BasinArrayPreDict = dict()
BasinArrayDefDict = dict()
for bas in BAS_KEYS:
    BasinArrayPreDict[bas] = dict()
    BasinArrayDefDict[bas] = dict()
    for mon in MonIndexer:
        BasinArrayPreDict[bas][mon] = list()
        BasinArrayDefDict[bas][mon] = list()
    # end for
# end for

In [29]:
for bas in BAS_KEYS:
    # do all models
    mDictDF = dict()
    mStatsDictDF = dict()
    for mKey in MODEL_KEYS:
        mVals = MODEL_DICT[mKey]
        cMiper = mVals[0]
        curSens = mVals[1]
        if cMiper == "CMIP6":
            inDir = IN_DIR3
        else:
            inDir = IN_DIR4
        # end if
        for tSen in curSens:
            if tSen in ["ssp585", "rcp85"]:
                mLabeler = "M%02dH 2031-2060" % mKey
            else:
                mLabeler = "M%02dL 2031-2060" % mKey
            # end if
            InFiler = os.path.normpath( os.path.join( inDir, ProjModelOutsName % (mKey, tSen, bas) ) )
            cModDDF = pd.read_pickle( InFiler )
            cModDDF["YrMo"] = cModDDF.apply( lambda row: setYrMo( row["Year"], row["Month"] ), axis=1 )
            cModDDF = cModDDF[(cModDDF["Year"] >= 2030) & (cModDDF["Year"] <= 2060)].copy()
            cMMonPre = cModDDF[["Pre_mm", "YrMo"]].groupby(by="YrMo").sum()
            cMMonDef = cModDDF[["Def_mm", "YrMo"]].groupby(by="YrMo").sum()
            cMMonPre["PreCS"] = cMMonPre["Pre_mm"].rolling(window=3,).sum()
            cMMonDef["DefCS"] = cMMonDef["Def_mm"].rolling(window=3,).sum()
            cMMonPre["Year"] = [ int( x / 100 ) for x in cMMonPre.index.to_list() ]
            cMMonDef["Year"] = [ int( x / 100 ) for x in cMMonDef.index.to_list() ]
            cMMonPre["Month"] = [ int( x - ( y * 100 ) ) for x,y in zip( cMMonPre.index.to_list(), cMMonPre["Year"].to_list() ) ]
            cMMonDef["Month"] = [ int( x - ( y * 100 ) ) for x,y in zip( cMMonDef.index.to_list(), cMMonDef["Year"].to_list() ) ]
            cMMonPre = cMMonPre[(cMMonPre["Year"] >= 2031) & (cMMonPre["Year"] <= 2060)].copy()
            cMMonDef = cMMonDef[(cMMonDef["Year"] >= 2031) & (cMMonDef["Year"] <= 2060)].copy()
            # go through each month
            PreM3MonDict = dict()
            DefM3MonDict = dict()
            for mon in MonIndexer:
                # SPI
                m3Mon = cMMonPre[cMMonPre["Month"] == mon].copy()
                a3Mon = m3Mon["PreCS"].to_numpy(dtype=np.float32)
                BasinArrayPreDict[bas][mon].append( a3Mon )
                extA3Mon = deepcopy( ObsCumPre[bas][mon] )
                fitT3Mon = sstats.pearson3.fit( a3Mon )
                lD3Mon = { "skew" : fitT3Mon[0], "location" : fitT3Mon[1],
                           "scale" : fitT3Mon[2], }
                # now get the cumulative probability values from the distributions
                #   for the observed values.
                pre3MCDF = sstats.pearson3.cdf( a3Mon, lD3Mon["skew"], loc=lD3Mon["location"], 
                                                scale=lD3Mon["scale"] )
                hpre3MCDF = sstats.pearson3.cdf( extA3Mon, lD3Mon["skew"], loc=lD3Mon["location"], 
                                                 scale=lD3Mon["scale"] )
                # make sure the cumulative density is not less than 1.0 / (2* N years).
                pre3MCDF = np.where( pre3MCDF < (1.0/60.0), (1.0/60.0), pre3MCDF )
                hpre3MCDF = np.where( hpre3MCDF < (1.0/60.0), (1.0/60.0), hpre3MCDF )
                SPI3Mo = sstats.norm.ppf( pre3MCDF, loc=ZMu, scale=ZStd )
                hSPI3Mo = sstats.norm.ppf( hpre3MCDF, loc=ZMu, scale=ZStd )
                # dictionary entries for this month
                PreM3MonDict[mon] = [ [pre3MCDF, hpre3MCDF, SPI3Mo, hSPI3Mo, a3Mon, extA3Mon], deepcopy( lD3Mon ) ]
                # SPEI
                m3Mon = cMMonDef[cMMonDef["Month"] == mon].copy()
                a3Mon = m3Mon["DefCS"].to_numpy(dtype=np.float32)
                BasinArrayDefDict[bas][mon].append( a3Mon )
                extA3Mon = deepcopy( ObsCumDef[bas][mon] )
                # fit 'generalized logistic' distributions to these arrays
                lD3Mon = estimatellogparams( a3Mon )
                # now get the cumulative probability values from the distributions
                #   for the observed values.
                hdef3MCDF = probDistLLogis( lD3Mon, extA3Mon )
                def3MCDF = probDistLLogis( lD3Mon, a3Mon )
                # make sure the cumulative density is not less than 1.0 / (2* N years).
                hdef3MCDF = np.where( hdef3MCDF < (1.0/60.0), (1.0/60.0), hdef3MCDF )
                def3MCDF = np.where( def3MCDF < (1.0/60.0), (1.0/60.0), def3MCDF )
                SPEI3Mo = sstats.norm.ppf( def3MCDF, loc=ZMu, scale=ZStd )
                hSPEI3Mo = sstats.norm.ppf( hdef3MCDF, loc=ZMu, scale=ZStd )
                # dictionary entries for this month
                DefM3MonDict[mon] = [ [def3MCDF, hdef3MCDF, SPEI3Mo, hSPEI3Mo, a3Mon, extA3Mon], deepcopy( lD3Mon ) ]
                # if the month is July then calculate our drought target probability
                if mon == 7:
                    # get our probability for the target cumulative deficit
                    curDTargProb = probDistLLogis( lD3Mon, np.array( [float(DroughtTargsDict[bas][2])], dtype=np.float32 ) )
                    curDTargProb = np.where( curDTargProb < (1.0/60.0), (1.0/60.0), curDTargProb )
                    # assign to drought targest dictionary
                    DroughtTargsDict[bas][3][mLabeler] = float( curDTargProb[0] )
                # end if
            # end month for
            # rebuild time series in calendar month format
            # rebuild time series representation from calendar month lists
            spi3mList = list()
            Hspi3mList = list()
            cpSPI3mList = list()
            HcpSPI3mList = list()
            cdSPI3mList = list()
            HcdSPI3mList = list()
            spei3mList = list()
            Hspei3mList = list()
            cpSPEI3mList = list()
            HcpSPEI3mList = list()
            cdSPEI3mList = list()
            HcdSPEI3mList = list()
            # loop through years and fill by month order
            for yI in range(30):
                for mI in range(1, 13, 1):
                    m3SPIArrayCP = PreM3MonDict[mI][0][0]
                    Hm3SPIArrayCP = PreM3MonDict[mI][0][1]
                    m3SPIArraySP = PreM3MonDict[mI][0][2]
                    Hm3SPIArraySP = PreM3MonDict[mI][0][3]
                    m3SPIArrayCD = PreM3MonDict[mI][0][4]
                    Hm3SPIArrayCD = PreM3MonDict[mI][0][5]
                    m3SPEIArrayCP = DefM3MonDict[mI][0][0]
                    Hm3SPEIArrayCP = DefM3MonDict[mI][0][1]
                    m3SPEIArraySP = DefM3MonDict[mI][0][2]
                    Hm3SPEIArraySP = DefM3MonDict[mI][0][3]
                    m3SPEIArrayCD = DefM3MonDict[mI][0][4]
                    Hm3SPEIArrayCD = DefM3MonDict[mI][0][5]
                    if yI < len( m3SPIArrayCP ):
                        spi3mList.append( m3SPIArraySP[yI] )
                        Hspi3mList.append( Hm3SPIArraySP[yI] )
                        cpSPI3mList.append( m3SPIArrayCP[yI] )
                        HcpSPI3mList.append( Hm3SPIArrayCP[yI] )
                        cdSPI3mList.append( m3SPIArrayCD[yI] )
                        HcdSPI3mList.append( Hm3SPIArrayCD[yI] )
                        spei3mList.append( m3SPEIArraySP[yI] )
                        Hspei3mList.append( Hm3SPEIArraySP[yI] )
                        cpSPEI3mList.append( m3SPEIArrayCP[yI] )
                        HcpSPEI3mList.append( Hm3SPEIArrayCP[yI] )
                        cdSPEI3mList.append( m3SPEIArrayCD[yI] )
                        HcdSPEI3mList.append( m3SPEIArrayCD[yI] )
                    # end if
                # end inner for
            # end outer for
            # now build the time indexed DataFrame
            DataDict = { "Year" : cMMonPre["Year"].to_numpy(dtype=np.int32),
                         "Month" : cMMonPre["Month"].to_numpy(dtype=np.int32),
                         "CumPre" : np.array( cdSPI3mList, dtype=np.float32 ),
                         "Hist_CumPre" : np.array( HcdSPI3mList, dtype=np.float32 ),
                         "CumPreProb" : np.array( cpSPI3mList, dtype=np.float32 ),
                         "Hist_CumPreProb" : np.array( HcpSPI3mList, dtype=np.float32 ),
                         "SPI" : np.array( spi3mList, dtype=np.float32 ),
                         "Hist_SPI" : np.array( Hspi3mList, dtype=np.float32 ),
                         "CumDef" : np.array( cdSPEI3mList, dtype=np.float32 ),
                         "Hist_CumDef" : np.array( HcdSPEI3mList, dtype=np.float32 ),
                         "CumDefProb" : np.array( cpSPEI3mList, dtype=np.float32 ), 
                         "Hist_CumDefProb" : np.array( HcpSPEI3mList, dtype=np.float32 ), 
                         "SPEI" : np.array( spei3mList, dtype=np.float32 ),
                         "Hist_SPEI" : np.array( Hspei3mList, dtype=np.float32 ), }
            cSP_3 = pd.DataFrame( index=cMMonPre.index, data=DataDict )
            mDictDF[mLabeler] = cSP_3.copy()
            # now do the stats
            stats3SPIList = list()
            stats3SPEIList = list()
            indINList = list()
            indENList = list()
            for mI in range(1, 13, 1):
                lDPre3Mon = PreM3MonDict[mI][1]
                indINList.append( "skew_%d" % mI )
                indINList.append( "scale_%d" % mI )
                indINList.append( "loc_%d" % mI )
                stats3SPIList.append( lDPre3Mon["skew"] )
                stats3SPIList.append( lDPre3Mon["scale"] )
                stats3SPIList.append( lDPre3Mon["location"] )
                lDDef3Mon = DefM3MonDict[mI][1]
                indENList.append( "shape_%d" % mI )
                indENList.append( "scale_%d" % mI )
                indENList.append( "loc_%d" % mI )
                stats3SPEIList.append( lDDef3Mon["k"] )
                stats3SPEIList.append( lDDef3Mon["scale"] )
                stats3SPEIList.append( lDDef3Mon["loc"] )
            # end for
            # build our DataFrames
            c3SPIStatsDF = pd.DataFrame( index=indINList, data={"SPI Fit Stats" : stats3SPIList,} )
            c3SPEIStatsDF = pd.DataFrame( index=indENList, data={"SPEI Fit Stats" : stats3SPEIList,} )
            mStatsDictDF[mLabeler] = [c3SPIStatsDF.copy(), c3SPEIStatsDF.copy()]
            # do our plots
            # SPEI
            bT3Mo = cSP_3[["Year", "Month", "Hist_SPEI"]].copy()
            bT3Mo["AltYear"] = bT3Mo["Year"] - 38
            pvbT3Mo = bT3Mo.pivot( index="AltYear", columns="Month", values="Hist_SPEI")
            NumYrs = len( pvbT3Mo )
            allSPEI_1 = pvbT3Mo.to_numpy( dtype=np.float32 )
            matLister1 = list()
            for iI in range( NumYrs ):
                rowLister = list()
                for jJ in range(12):
                    cVal = allSPEI_1[iI, jJ]
                    if ( cVal >= 1.5 ) or ( cVal <= -1.5 ):
                        rowLister.append( "%4.1f" % cVal )
                    else:
                        rowLister.append( "" )
                    # end if
                # end inner for
                matLister1.append( rowLister )
            # end outer for
            AnnotMat_1 = np.array( matLister1 )
            OutFilePDF = os.path.normpath( os.path.join( OUT_DIR, "Plots", "%s_%s_SPEI_3mon-1993to2022.pdf" % (bas, mLabeler) ) )
            OutFileSVG = os.path.normpath( os.path.join( OUT_DIR, "Plots", "%s_%s_SPEI_3mon-1993to2022.svg" % (bas, mLabeler) ) )
            OutFilePNG = os.path.normpath( os.path.join( OUT_DIR, "Plots", "%s_%s_SPEI_3mon-1993to2022.png" % (bas, mLabeler) ) )
            Fig1 = plt.figure()
            Fig1.set_size_inches(7.5, 10.5)
            ax11 = Fig1.add_subplot(111)
            ax11 = sns.heatmap( pvbT3Mo, vmin=pMin, vmax=pMax, cmap=SegCMap, center=0.0,
                                annot=AnnotMat_1, fmt="s", linecolor="gainsboro", linewidths=0.0,
                                annot_kws={'fontsize':9, 'color':'xkcd:black'},
                                cbar_kws={'label': 'SPEI',}, ax=ax11 )
            cbar = ax11.collections[0].colorbar
            cbar.ax.tick_params(labelsize=9)
            ax11.set_title( "%s Basin – %s, 3-month SPEI (2031-2060 CN)" % (bas, mLabeler), fontsize=12 )
            ax11.set_xlabel('Month', fontsize=10 )
            ax11.set_ylabel('Year', fontsize=10)
            ax11.tick_params(axis='both', which='major', labelsize=10)
            Fig1.savefig( OutFileSVG, dpi=600 )
            Fig1.savefig( OutFilePNG, dpi=600 )
            Fig1.savefig( OutFilePDF, dpi=600 )
            # clear the figures
            plt.cla()
            plt.close(Fig1)
            # SPI
            bT3Mo = cSP_3[["Year", "Month", "Hist_SPI"]].copy()
            bT3Mo["AltYear"] = bT3Mo["Year"] - 38
            pvbT3Mo = bT3Mo.pivot( index="AltYear", columns="Month", values="Hist_SPI")
            NumYrs = len( pvbT3Mo )
            allSPEI_1 = pvbT3Mo.to_numpy( dtype=np.float32 )
            matLister1 = list()
            for iI in range( NumYrs ):
                rowLister = list()
                for jJ in range(12):
                    cVal = allSPEI_1[iI, jJ]
                    if ( cVal >= 1.5 ) or ( cVal <= -1.5 ):
                        rowLister.append( "%4.1f" % cVal )
                    else:
                        rowLister.append( "" )
                    # end if
                # end inner for
                matLister1.append( rowLister )
            # end outer for
            AnnotMat_1 = np.array( matLister1 )
            OutFilePDF = os.path.normpath( os.path.join( OUT_DIR, "Plots", "%s_%s_SPI_3mon-1993to2022.pdf" % (bas, mLabeler) ) )
            OutFileSVG = os.path.normpath( os.path.join( OUT_DIR, "Plots", "%s_%s_SPI_3mon-1993to2022.svg" % (bas, mLabeler) ) )
            OutFilePNG = os.path.normpath( os.path.join( OUT_DIR, "Plots", "%s_%s_SPI_3mon-1993to2022.png" % (bas, mLabeler) ) )
            Fig1 = plt.figure()
            Fig1.set_size_inches(7.5, 10.5)
            ax11 = Fig1.add_subplot(111)
            ax11 = sns.heatmap( pvbT3Mo, vmin=pMin, vmax=pMax, cmap=SegCMap, center=0.0,
                                annot=AnnotMat_1, fmt="s", linecolor="gainsboro", linewidths=0.0,
                                annot_kws={'fontsize':9, 'color':'xkcd:black'},
                                cbar_kws={'label': 'SPI',}, ax=ax11 )
            cbar = ax11.collections[0].colorbar
            cbar.ax.tick_params(labelsize=9)
            ax11.set_title( "%s Basin – %s, 3-month SPI (2031-2060 CN)" % (bas, mLabeler), fontsize=12 )
            ax11.set_xlabel('Month', fontsize=10 )
            ax11.set_ylabel('Year', fontsize=10)
            ax11.tick_params(axis='both', which='major', labelsize=10)
            Fig1.savefig( OutFileSVG, dpi=600 )
            Fig1.savefig( OutFilePNG, dpi=600 )
            Fig1.savefig( OutFilePDF, dpi=600 )
            # clear the figures
            plt.cla()
            plt.close(Fig1)
        # end scenario for
    # end model for
    # output stuff that needs to be output for this basin
    outXLSX = os.path.normpath( os.path.join( OUT_DIR, "%s_SPEI_and_SPI.xlsx" % bas ) )
    writer = pd.ExcelWriter( outXLSX )
    workbook  = writer.book
    format1 = workbook.add_format({'num_format': '#,##0.00'})
    allKeys = list( mDictDF.keys() )
    for mLabel in allKeys:
        cSPIStats = mStatsDictDF[mLabel][0]
        cSPEIStats = mStatsDictDF[mLabel][1]
        curDF = mDictDF[mLabel]
        cLabel = "%s_SPI_Info" % mLabel
        cSPIStats.to_excel( writer, sheet_name=cLabel, index_label="Parameter" )
        # adjust columns
        writer.sheets[cLabel].set_column( 0, 0, 15 )
        for column in cSPIStats:
            column_width = max(cSPIStats[column].astype(str).map(len).max()+6, len(column)+6)
            col_idx = cSPIStats.columns.get_loc(column)
            writer.sheets[cLabel].set_column(col_idx+1, col_idx+1, column_width, format1)
        # end for
        cLabel = "%s_SPEI_Info" % mLabel
        cSPEIStats.to_excel( writer, sheet_name=cLabel, index_label="Parameter" )
        # adjust columns
        writer.sheets[cLabel].set_column( 0, 0, 15 )
        for column in cSPEIStats:
            column_width = max(cSPEIStats[column].astype(str).map(len).max()+6, len(column)+6)
            col_idx = cSPEIStats.columns.get_loc(column)
            writer.sheets[cLabel].set_column(col_idx+1, col_idx+1, column_width, format1)
        # end for
        cLabel = "%s_Results" % mLabel
        curDF.to_excel( writer, sheet_name=cLabel, )
        # adjust columns
        writer.sheets[cLabel].set_column( 0, 0, 15 )
        for column in curDF:
            column_width = max(curDF[column].astype(str).map(len).max()+6, len(column)+6)
            col_idx = curDF.columns.get_loc(column)
            writer.sheets[cLabel].set_column(col_idx+1, col_idx+1, column_width, format1)
        # end for
    # end model out for
    writer.close()
# end basin for

## Do Ensemble Calcs

The full basin calcs aggregate the individual model runs into an ensemble analysis

In [30]:
npList = BasinArrayPreDict[BAS_KEYS[0]][7]
len(npList)

19

In [31]:
testA = np.concatenate( npList )
testA.shape

(570,)

In [32]:
DictDF = dict()
StatsDictDF = dict()

In [33]:
for bas in BAS_KEYS:
    # only work on basins
    PreM3MonDict = dict()
    DefM3MonDict = dict()
    for mon in MonIndexer:
        cMonPreList = BasinArrayPreDict[bas][mon]
        cMonPre = np.concatenate( cMonPreList )
        cMonDefList = BasinArrayDefDict[bas][mon]
        cMonDef = np.concatenate( cMonDefList )
        # SPI
        a3Mon = cMonPre
        extA3Mon = deepcopy( ObsCumPre[bas][mon] )
        fitT3Mon = sstats.pearson3.fit( a3Mon )
        lD3Mon = { "skew" : fitT3Mon[0], "location" : fitT3Mon[1],
                   "scale" : fitT3Mon[2], }
        # now get the cumulative probability values from the distributions
        #   for the observed values.
        pre3MCDF = sstats.pearson3.cdf( a3Mon, lD3Mon["skew"], loc=lD3Mon["location"], 
                                        scale=lD3Mon["scale"] )
        hpre3MCDF = sstats.pearson3.cdf( extA3Mon, lD3Mon["skew"], loc=lD3Mon["location"], 
                                         scale=lD3Mon["scale"] )
        # make sure the cumulative density is not less than 1.0 / (2* N years).
        pre3MCDF = np.where( pre3MCDF < (1.0/60.0), (1.0/60.0), pre3MCDF )
        hpre3MCDF = np.where( hpre3MCDF < (1.0/60.0), (1.0/60.0), hpre3MCDF )
        SPI3Mo = sstats.norm.ppf( pre3MCDF, loc=ZMu, scale=ZStd )
        hSPI3Mo = sstats.norm.ppf( hpre3MCDF, loc=ZMu, scale=ZStd )
        # dictionary entries for this month
        PreM3MonDict[mon] = [ [pre3MCDF, hpre3MCDF, SPI3Mo, hSPI3Mo, a3Mon, extA3Mon], deepcopy( lD3Mon ) ]
        # SPEI
        a3Mon = cMonDef
        extA3Mon = deepcopy( ObsCumDef[bas][mon] )
        # fit 'generalized logistic' distributions to these arrays
        lD3Mon = estimatellogparams( a3Mon )
        # now get the cumulative probability values from the distributions
        #   for the observed values.
        hdef3MCDF = probDistLLogis( lD3Mon, extA3Mon )
        def3MCDF = probDistLLogis( lD3Mon, a3Mon )
        # make sure the cumulative density is not less than 1.0 / (2* N years).
        hdef3MCDF = np.where( hdef3MCDF < (1.0/60.0), (1.0/60.0), hdef3MCDF )
        def3MCDF = np.where( def3MCDF < (1.0/60.0), (1.0/60.0), def3MCDF )
        SPEI3Mo = sstats.norm.ppf( def3MCDF, loc=ZMu, scale=ZStd )
        hSPEI3Mo = sstats.norm.ppf( hdef3MCDF, loc=ZMu, scale=ZStd )
        # dictionary entries for this month
        DefM3MonDict[mon] = [ [def3MCDF, hdef3MCDF, SPEI3Mo, hSPEI3Mo, a3Mon, extA3Mon], deepcopy( lD3Mon ) ]
        # if the month is July then calculate our drought target probability
        if mon == 7:
            # get our probability for the target cumulative deficit
            curDTargProb = probDistLLogis( lD3Mon, np.array( [float(DroughtTargsDict[bas][2])], dtype=np.float32 ) )
            curDTargProb = np.where( curDTargProb < (1.0/60.0), (1.0/60.0), curDTargProb )
            # assign to drought targest dictionary
            DroughtTargsDict[bas][3]["ensemble"] = float( curDTargProb[0] )
        # end if
    # end month for
    # rebuild time series representation from calendar month lists
    spi3mList = list()
    Hspi3mList = list()
    cpSPI3mList = list()
    HcpSPI3mList = list()
    cdSPI3mList = list()
    HcdSPI3mList = list()
    spei3mList = list()
    Hspei3mList = list()
    cpSPEI3mList = list()
    HcpSPEI3mList = list()
    cdSPEI3mList = list()
    HcdSPEI3mList = list()
    # loop through years and fill by month order
    for yI in range(30):
        for mI in range(1, 13, 1):
            m3SPIArrayCP = PreM3MonDict[mI][0][0]
            Hm3SPIArrayCP = PreM3MonDict[mI][0][1]
            m3SPIArraySP = PreM3MonDict[mI][0][2]
            Hm3SPIArraySP = PreM3MonDict[mI][0][3]
            m3SPIArrayCD = PreM3MonDict[mI][0][4]
            Hm3SPIArrayCD = PreM3MonDict[mI][0][5]
            m3SPEIArrayCP = DefM3MonDict[mI][0][0]
            Hm3SPEIArrayCP = DefM3MonDict[mI][0][1]
            m3SPEIArraySP = DefM3MonDict[mI][0][2]
            Hm3SPEIArraySP = DefM3MonDict[mI][0][3]
            m3SPEIArrayCD = DefM3MonDict[mI][0][4]
            Hm3SPEIArrayCD = DefM3MonDict[mI][0][5]
            if yI < len( m3SPIArrayCP ):
                spi3mList.append( m3SPIArraySP[yI] )
                Hspi3mList.append( Hm3SPIArraySP[yI] )
                cpSPI3mList.append( m3SPIArrayCP[yI] )
                HcpSPI3mList.append( Hm3SPIArrayCP[yI] )
                cdSPI3mList.append( m3SPIArrayCD[yI] )
                HcdSPI3mList.append( Hm3SPIArrayCD[yI] )
                spei3mList.append( m3SPEIArraySP[yI] )
                Hspei3mList.append( Hm3SPEIArraySP[yI] )
                cpSPEI3mList.append( m3SPEIArrayCP[yI] )
                HcpSPEI3mList.append( Hm3SPEIArrayCP[yI] )
                cdSPEI3mList.append( m3SPEIArrayCD[yI] )
                HcdSPEI3mList.append( m3SPEIArrayCD[yI] )
            # end if
        # end inner for
    # end outer for
    # now build the time indexed DataFrame
    DataDict = { "Year" : cMMonPre["Year"].to_numpy(dtype=np.int32),
                 "Month" : cMMonPre["Month"].to_numpy(dtype=np.int32),
                 "CumPre" : np.array( cdSPI3mList, dtype=np.float32 ),
                 "Hist_CumPre" : np.array( HcdSPI3mList, dtype=np.float32 ),
                 "CumPreProb" : np.array( cpSPI3mList, dtype=np.float32 ),
                 "Hist_CumPreProb" : np.array( HcpSPI3mList, dtype=np.float32 ),
                 "SPI" : np.array( spi3mList, dtype=np.float32 ),
                 "Hist_SPI" : np.array( Hspi3mList, dtype=np.float32 ),
                 "CumDef" : np.array( cdSPEI3mList, dtype=np.float32 ),
                 "Hist_CumDef" : np.array( HcdSPEI3mList, dtype=np.float32 ),
                 "CumDefProb" : np.array( cpSPEI3mList, dtype=np.float32 ), 
                 "Hist_CumDefProb" : np.array( HcpSPEI3mList, dtype=np.float32 ), 
                 "SPEI" : np.array( spei3mList, dtype=np.float32 ),
                 "Hist_SPEI" : np.array( Hspei3mList, dtype=np.float32 ), }
    cSP_3 = pd.DataFrame( index=cMMonPre.index, data=DataDict )
    DictDF[bas] = cSP_3.copy()
    # now do the stats
    stats3SPIList = list()
    stats3SPEIList = list()
    indINList = list()
    indENList = list()
    for mI in range(1, 13, 1):
        lDPre3Mon = PreM3MonDict[mI][1]
        indINList.append( "skew_%d" % mI )
        indINList.append( "scale_%d" % mI )
        indINList.append( "loc_%d" % mI )
        stats3SPIList.append( lDPre3Mon["skew"] )
        stats3SPIList.append( lDPre3Mon["scale"] )
        stats3SPIList.append( lDPre3Mon["location"] )
        lDDef3Mon = DefM3MonDict[mI][1]
        indENList.append( "shape_%d" % mI )
        indENList.append( "scale_%d" % mI )
        indENList.append( "loc_%d" % mI )
        stats3SPEIList.append( lDDef3Mon["k"] )
        stats3SPEIList.append( lDDef3Mon["scale"] )
        stats3SPEIList.append( lDDef3Mon["loc"] )
    # end for
    # build our DataFrames
    c3SPIStatsDF = pd.DataFrame( index=indINList, data={"SPI Fit Stats" : stats3SPIList,} )
    c3SPEIStatsDF = pd.DataFrame( index=indENList, data={"SPEI Fit Stats" : stats3SPEIList,} )
    StatsDictDF[bas] = [c3SPIStatsDF.copy(), c3SPEIStatsDF.copy()]
    # do our plots
    # SPEI
    bT3Mo = cSP_3[["Year", "Month", "Hist_SPEI"]].copy()
    bT3Mo["AltYear"] = bT3Mo["Year"] - 38
    pvbT3Mo = bT3Mo.pivot( index="AltYear", columns="Month", values="Hist_SPEI")
    NumYrs = len( pvbT3Mo )
    allSPEI_1 = pvbT3Mo.to_numpy( dtype=np.float32 )
    matLister1 = list()
    for iI in range( NumYrs ):
        rowLister = list()
        for jJ in range(12):
            cVal = allSPEI_1[iI, jJ]
            if ( cVal >= 1.5 ) or ( cVal <= -1.5 ):
                rowLister.append( "%4.1f" % cVal )
            else:
                rowLister.append( "" )
            # end if
        # end inner for
        matLister1.append( rowLister )
    # end outer for
    AnnotMat_1 = np.array( matLister1 )
    OutFilePDF = os.path.normpath( os.path.join( OUT_DIR, "Plots", "FBe-%s_SPEI_3mon-1993to2022.pdf" % bas ) )
    OutFileSVG = os.path.normpath( os.path.join( OUT_DIR, "Plots", "FBe-%s_SPEI_3mon-1993to2022.svg" % bas ) )
    OutFilePNG = os.path.normpath( os.path.join( OUT_DIR, "Plots", "FBe-%s_SPEI_3mon-1993to2022.png" % bas ) )
    Fig1 = plt.figure()
    Fig1.set_size_inches(7.5, 10.5)
    ax11 = Fig1.add_subplot(111)
    ax11 = sns.heatmap( pvbT3Mo, vmin=pMin, vmax=pMax, cmap=SegCMap, center=0.0,
                        annot=AnnotMat_1, fmt="s", linecolor="gainsboro", linewidths=0.0,
                        annot_kws={'fontsize':9, 'color':'xkcd:black'},
                        cbar_kws={'label': 'SPEI',}, ax=ax11 )
    cbar = ax11.collections[0].colorbar
    cbar.ax.tick_params(labelsize=9)
    ax11.set_title( "%s Ensemble, 3-month SPEI (2031-2060 CN)" % bas, fontsize=12 )
    ax11.set_xlabel('Month', fontsize=10 )
    ax11.set_ylabel('Year', fontsize=10)
    ax11.tick_params(axis='both', which='major', labelsize=10)
    Fig1.savefig( OutFileSVG, dpi=600 )
    Fig1.savefig( OutFilePNG, dpi=600 )
    Fig1.savefig( OutFilePDF, dpi=600 )
    # clear the figures
    plt.cla()
    plt.close(Fig1)
    # SPI
    bT3Mo = cSP_3[["Year", "Month", "Hist_SPI"]].copy()
    bT3Mo["AltYear"] = bT3Mo["Year"] - 38
    pvbT3Mo = bT3Mo.pivot( index="AltYear", columns="Month", values="Hist_SPI")
    NumYrs = len( pvbT3Mo )
    allSPEI_1 = pvbT3Mo.to_numpy( dtype=np.float32 )
    matLister1 = list()
    for iI in range( NumYrs ):
        rowLister = list()
        for jJ in range(12):
            cVal = allSPEI_1[iI, jJ]
            if ( cVal >= 1.5 ) or ( cVal <= -1.5 ):
                rowLister.append( "%4.1f" % cVal )
            else:
                rowLister.append( "" )
            # end if
        # end inner for
        matLister1.append( rowLister )
    # end outer for
    AnnotMat_1 = np.array( matLister1 )
    OutFilePDF = os.path.normpath( os.path.join( OUT_DIR, "Plots", "FBe-%s_SPI_3mon-1993to2022.pdf" % bas ) )
    OutFileSVG = os.path.normpath( os.path.join( OUT_DIR, "Plots", "FBe-%s_SPI_3mon-1993to2022.svg" % bas ) )
    OutFilePNG = os.path.normpath( os.path.join( OUT_DIR, "Plots", "FBe-%s_SPI_3mon-1993to2022.png" % bas ) )
    Fig1 = plt.figure()
    Fig1.set_size_inches(7.5, 10.5)
    ax11 = Fig1.add_subplot(111)
    ax11 = sns.heatmap( pvbT3Mo, vmin=pMin, vmax=pMax, cmap=SegCMap, center=0.0,
                        annot=AnnotMat_1, fmt="s", linecolor="gainsboro", linewidths=0.0,
                        annot_kws={'fontsize':9, 'color':'xkcd:black'},
                        cbar_kws={'label': 'SPI',}, ax=ax11 )
    cbar = ax11.collections[0].colorbar
    cbar.ax.tick_params(labelsize=9)
    ax11.set_title( "%s Ensemble, 3-month SPI (2031-2060 CN)" % bas, fontsize=12 )
    ax11.set_xlabel('Month', fontsize=10 )
    ax11.set_ylabel('Year', fontsize=10)
    ax11.tick_params(axis='both', which='major', labelsize=10)
    Fig1.savefig( OutFileSVG, dpi=600 )
    Fig1.savefig( OutFilePNG, dpi=600 )
    Fig1.savefig( OutFilePDF, dpi=600 )
    # clear the figures
    plt.cla()
    plt.close(Fig1)
# end basin for

Collate the drought target information into a DataFrame

In [34]:
allKeys = sorted( DroughtTargsDict[BAS_KEYS[0]][3].keys() )
len( allKeys )

20

In [35]:
mAllKeys = [ "ensemble", 'M02H 2031-2060', 'M02L 2031-2060', 'M03H 2031-2060', 'M03L 2031-2060', 'M04H 2031-2060', 'M04L 2031-2060',
             'M05H 2031-2060', 'M05L 2031-2060', 'M07H 2031-2060', 'M07L 2031-2060', 'M08H 2031-2060', 'M08L 2031-2060',
             'M09H 2031-2060', 'M09L 2031-2060', 'M10H 2031-2060', 'M11H 2031-2060', 'M11L 2031-2060', 'M12H 2031-2060',
             'M12L 2031-2060', ]
len( mAllKeys )

20

In [36]:
outIndexList = [ "Jul-2022 3-month cum. deficit", "Jul-2022 SPEI cum. prob.", "5X cum. prob.",]
outIndexList.extend( mAllKeys )
len( outIndexList )

23

In [37]:
DataDict = dict()
for bas in BAS_KEYS:
    outLister = list()
    outLister.append( DroughtTargsDict[bas][2] )
    curCumProb = DroughtTargsDict[bas][1]
    curCP5X = 5.0*curCumProb
    outLister.append( curCumProb )
    outLister.append( curCP5X )
    for cAKey in mAllKeys:
        outLister.append( DroughtTargsDict[bas][3][cAKey] )
    # end for
    DataDict[bas] = np.array( outLister, dtype=np.float32 )
# end for

In [38]:
DTSummDF = pd.DataFrame( index=outIndexList, data=DataDict )

In [39]:
display( HTML( DTSummDF.to_html() ) )

,Blanco,Cibolo,Frio,Guadalupe,Med-Cib,Medina,Nueces,Sab-Med,Sabinal
Jul-2022 3-month cum. deficit,-534.390198,-533.064636,-539.547729,-528.912842,-549.221680,-542.801514,-538.581970,-543.716736,-531.497803
Jul-2022 SPEI cum. prob.,0.034969,0.042354,0.029823,0.027471,0.040599,0.027134,0.046460,0.030477,0.040542
5X cum. prob.,0.174845,0.211768,0.149113,0.137354,0.202995,0.135670,0.232299,0.152384,0.202711
ensemble,0.056472,0.058086,0.067510,0.062759,0.046603,0.054027,0.090919,0.066677,0.081483
M02H 2031-2060,0.158851,0.163343,0.137907,0.157622,0.144155,0.143721,0.189095,0.166658,0.178190
M02L 2031-2060,0.048966,0.051991,0.042243,0.039331,0.043531,0.032195,0.069912,0.048467,0.057109
M03H 2031-2060,0.029702,0.019736,0.016667,0.022762,0.016667,0.016667,0.016667,0.016667,0.016906
M03L 2031-2060,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667
M04H 2031-2060,0.087769,0.084264,0.107331,0.099353,0.065177,0.077224,0.135856,0.100719,0.123767
M04L 2031-2060,0.060246,0.062760,0.095130,0.077900,0.049658,0.068429,0.128826,0.088625,0.107862


Output to a summary spreadsheet

In [40]:
outXLSX = os.path.normpath( os.path.join( OUT_DIR, "Basin_Ensembles_Summaries.xlsx" ) )
writer = pd.ExcelWriter( outXLSX )
workbook  = writer.book
format1 = workbook.add_format({'num_format': '#,##0.00'})
cLabel = "Drought Targets Summary"
DTSummDF.to_excel( writer, sheet_name=cLabel, )
writer.sheets[cLabel].set_column( 0, 0, 20 )
for column in DTSummDF:
    column_width = max(DTSummDF[column].astype(str).map(len).max()+6, len(column)+6)
    col_idx = DTSummDF.columns.get_loc(column)
    writer.sheets[cLabel].set_column(col_idx+1, col_idx+1, column_width, format1)
# end for
for bas in BAS_KEYS:
    curDF = DictDF[bas]
    cSPIStats = StatsDictDF[bas][0]
    cSPEIStats = StatsDictDF[bas][1]
    cLabel = "%s_SPI_Info" % bas
    cSPIStats.to_excel( writer, sheet_name=cLabel, index_label="Parameter" )
    # adjust columns
    writer.sheets[cLabel].set_column( 0, 0, 15 )
    for column in cSPIStats:
        column_width = max(cSPIStats[column].astype(str).map(len).max()+6, len(column)+6)
        col_idx = cSPIStats.columns.get_loc(column)
        writer.sheets[cLabel].set_column(col_idx+1, col_idx+1, column_width, format1)
    # end for
    cLabel = "%s_SPEI_Info" % bas
    cSPEIStats.to_excel( writer, sheet_name=cLabel, index_label="Parameter" )
    # adjust columns
    writer.sheets[cLabel].set_column( 0, 0, 15 )
    for column in cSPEIStats:
        column_width = max(cSPEIStats[column].astype(str).map(len).max()+6, len(column)+6)
        col_idx = cSPEIStats.columns.get_loc(column)
        writer.sheets[cLabel].set_column(col_idx+1, col_idx+1, column_width, format1)
    # end for
    cLabel = "%s_Results" % bas
    curDF.to_excel( writer, sheet_name=cLabel, )
    # adjust columns
    writer.sheets[cLabel].set_column( 0, 0, 15 )
    for column in curDF:
        column_width = max(curDF[column].astype(str).map(len).max()+6, len(column)+6)
        col_idx = curDF.columns.get_loc(column)
        writer.sheets[cLabel].set_column(col_idx+1, col_idx+1, column_width, format1)
    # end for
# end basin for
writer.close()